In [1]:
import pandas
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from openai import OpenAI
import utils

In [2]:
disease='MCTD'
symp_type='formal'
#model_old='chatgpt-3.5'
#model_new='chatgpt-4o'
#model_new_name = 'gpt-4o'


model_old='claude-3-opus'
model_new='claude-3.5-sonnet'
model_new_name = 'claude-3-5-sonnet-20240620'

filename=f'data/{disease}-{symp_type}-{model_old}.txt'
output_filename = f'data/{disease}-{symp_type}-{model_new}.txt'
keywords_list = ['mctd', 'mixed connective tissue disease']

## Run More Complex Models

The goal is to run more complex models and compare against the outputs of the baseline model.

In [3]:

all_patients = utils.process_file(filename)

Patient: number 1, gender female, age 10, weight 54, height 46
Symptom: Arthritis
Symptom: Gottron's papules
Symptom: Abnormal esophageal motility
Symptom: Cardiac disease
Symptom: Muscle disease
response: 1,*Juvenile Dermatomyositis*,70%,*The combination of arthritis, Gottron's papules (characteristic rash), abnormal esophageal motility, cardiac involvement, and muscle disease is highly suggestive of juvenile dermatomyositis, an autoimmune inflammatory disorder affecting muscles and skin.*
response: 2,*Mixed Connective Tissue Disease*,15%,*The presence of arthritis, muscle disease, and potential cardiac involvement could indicate mixed connective tissue disease, an overlap syndrome with features of various autoimmune disorders.*
response: 3,*Systemic Lupus Erythematosus*,10%,*Arthritis, muscle disease, and potential cardiac involvement can also be seen in systemic lupus erythematosus, an autoimmune disorder that can affect multiple organ systems.*
response: 4,*Scleroderma*,3%,*Abnorma

In [4]:
utils.run_llm_and_parse_response(all_patients[5], model_new_name)

Patient ID: 6


[('Systemic Lupus Erythematosus (SLE)',
  '40',
  " Juvenile-onset SLE can present with multiple system involvement including arthritis, Raynaud's phenomenon, fever, cardiac disease, and splenomegaly."),
 ('Juvenile Idiopathic Arthritis (JIA)',
  '25',
  " This condition commonly causes arthritis in children and can be associated with fever, cardiac involvement (pericarditis), and occasionally Raynaud's phenomenon."),
 ('Mixed Connective Tissue Disease',
  '15',
  " This overlap syndrome can present with features of lupus, scleroderma, and myositis, explaining the arthritis, Raynaud's phenomenon, and potential cardiac involvement."),
 ('Kawasaki Disease',
  '10',
  ' Although typically affecting younger children, it can cause fever, arthritis, cardiac disease (coronary artery involvement), and occasionally splenomegaly.'),
 ('Juvenile Dermatomyositis',
  '5',
  " This inflammatory myopathy can present with arthritis, Raynaud's phenomenon, fever, and rarely cardiac involvement, though m

In [5]:
resps = [ utils.run_llm_and_parse_response(p, model_new_name) for p in all_patients]

Patient ID: 1
Patient ID: 2
Patient ID: 3
Patient ID: 4
Patient ID: 5
Patient ID: 6
Patient ID: 7
Patient ID: 8
Patient ID: 9
Patient ID: 10
Patient ID: 11
Patient ID: 12
Patient ID: 13
Patient ID: 14
Patient ID: 15
Patient ID: 16
Patient ID: 17
Patient ID: 18
Patient ID: 19
Patient ID: 20
Patient ID: 21
Patient ID: 22
Patient ID: 23
Patient ID: 24
Patient ID: 25
Patient ID: 26
Patient ID: 27
Patient ID: 28
Patient ID: 29
Patient ID: 30
Patient ID: 31
Patient ID: 32
Patient ID: 33
Patient ID: 34
Patient ID: 35
Patient ID: 36
Patient ID: 37
Patient ID: 38
Patient ID: 39
Patient ID: 40
Patient ID: 41
Patient ID: 42
Patient ID: 43
Patient ID: 44
Patient ID: 45
Patient ID: 46
Patient ID: 47
Patient ID: 48
Patient ID: 49
Patient ID: 50
Patient ID: 51
Patient ID: 52
Patient ID: 53
Patient ID: 54
Patient ID: 55
Patient ID: 56
Patient ID: 57
Patient ID: 58
Patient ID: 59
Patient ID: 60
Patient ID: 61
Patient ID: 62
Patient ID: 63
Patient ID: 64
Patient ID: 65
Patient ID: 66
Patient ID: 67
Pati

In [6]:
print(resps)

[[('Juvenile Dermatomyositis', '80', " Presence of Gottron's papules and muscle disease are hallmark symptoms; arthritis and cardiac involvement are common"), ('Systemic Lupus Erythematosus (SLE)', '60', " Can present with arthritis, cardiac involvement, and muscle weakness; skin manifestations may mimic Gottron's papules"), ('Mixed Connective Tissue Disease', '40', ' Combines features of various autoimmune diseases, explaining the diverse symptoms including arthritis and muscle disease'), ('Scleroderma', '30', " Can cause esophageal dysmotility, cardiac involvement, and arthritis; skin changes may resemble Gottron's papules"), ('Polymyositis', '20', " Primarily causes muscle disease and can be associated with arthritis and cardiac involvement, but typically lacks skin manifestations like Gottron's papules")], [('Juvenile Dermatomyositis', '70', " Presence of Gottron's papules, muscle disease, and malar rash are highly characteristic; arthritis and Raynaud's phenomenon also support thi

In [7]:

def dump_response_to_file(filehandle, patient, resp):
    print(f'P: {patient.n}, {patient.gender}, {patient.age}, {patient.weight}, {patient.height}', file=filehandle)
    for s in patient.symptom_list:
        print(f'S:{s}', file=filehandle)
    for (j, r) in enumerate(resp):
        (disease_name, prob, descr) = r
        print(f'R: {j+1}, {disease_name}, {prob}, {descr}', file=filehandle)

# dump responses to output file
fhandle = open(output_filename,'w')
for (pt, resp) in zip(all_patients, resps):
    dump_response_to_file(fhandle, pt, resp)
fhandle.close()

In [8]:
def analyze_responses_for_top_three(resps, keywords_list, k=3):
    num_matches = 0
    for resp in resps:
        for (j, r) in enumerate(resp):
            if (j < k):
                (disease_name, prob, descr) = r
                if utils.approx_matches(keywords_list, [disease_name]):
                    num_matches = num_matches + 1
                    break 
    return num_matches

In [9]:
matches_1 = analyze_responses_for_top_three(resps, keywords_list, k=1)
print(f'{model_new} is able to find {disease} in top 1 for {matches_1} out of {len(resps)} cases')
matches_3 = analyze_responses_for_top_three(resps, keywords_list, k=3)
print(f'{model_new} is able to find {disease} in top 3 for {matches_3} out of {len(resps)} cases ')

claude-3.5-sonnet is able to find MCTD in top 1 for 6 out of 249 cases
claude-3.5-sonnet is able to find MCTD in top 3 for 245 out of 249 cases 


In [10]:
not_in_top_3 = utils.select_where_x_is_not_top(all_patients, keywords_list)
print(len(not_in_top_3))

11


In [11]:
chat_gpt_4_responses = [resps[pt.n-1] for pt in not_in_top_3]
matches_1 = analyze_responses_for_top_three(chat_gpt_4_responses, keywords_list, k=1)
print(f'{model_new} is able to find {disease} in top 1 for {matches_1} out of {len(not_in_top_3)} cases where {model_old}')
matches_3 = analyze_responses_for_top_three(chat_gpt_4_responses, keywords_list, k=3)
print(f'{model_new} is able to find {disease} in top 3 for {matches_3} out of {len(not_in_top_3)} cases where {model_old} ')

claude-3.5-sonnet is able to find MCTD in top 1 for 0 out of 11 cases where claude-3-opus
claude-3.5-sonnet is able to find MCTD in top 3 for 9 out of 11 cases where claude-3-opus 
